# Restaurant Agent Red Team Evaluation

This notebook performs red team evaluation on the Scheibmeir's restaurant agent to assess its safety and resilience against adversarial attacks.
It uses Azure AI Evaluation's Red Team functionality to test the agent across different risk categories.

## Setup and Configuration

In [1]:
import os
import asyncio
from typing import Dict, List, Optional, Any
from dotenv import load_dotenv
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

# Load environment variables from .env file
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Configuration from environment variables
RESTAURANT_ASSISTANT_ID = os.getenv("RESTAURANT_ASSISTANT_ID")
RESTAURANT_ASSISTANT_PROJECT = os.getenv("RESTAURANT_ASSISTANT_PROJECT")

print(f"Restaurant Assistant ID: {RESTAURANT_ASSISTANT_ID}")
print(f"Restaurant Project URL: {RESTAURANT_ASSISTANT_PROJECT}")

Restaurant Assistant ID: asst_eJv1oQY8pHlj3mgXyV2sLCYG
Restaurant Project URL: https://aipmaker-project-resource.services.ai.azure.com/api/projects/aipmaker-project


## Initialize Clients

In [3]:
# Initialize Azure credential
credential = DefaultAzureCredential()

# Initialize the Agents client
agents_client = AgentsClient(
    endpoint=RESTAURANT_ASSISTANT_PROJECT,
    credential=credential
)

print("Clients initialized successfully!")

Clients initialized successfully!


## Define Target Callback for Red Team Testing

This callback function will be used by the Red Team to test the restaurant agent.

In [4]:
async def restaurant_agent_callback(
    messages: list,
    stream: Optional[bool] = False,
    session_state: Optional[str] = None,
    context: Optional[Dict[str, Any]] = None,
) -> Dict[str, List[Dict[str, str]]]:
    """
    Callback function that interfaces with the restaurant agent.
    This function will be called by the Red Team evaluation framework.
    """
    try:
        # Extract the latest message from the conversation history
        latest_message = messages[-1].content if messages else "Hello"
        
        # Create a thread and run in one step
        result = agents_client.create_thread_and_process_run(
            agent_id=RESTAURANT_ASSISTANT_ID,
            thread={
                "messages": [
                    {
                        "role": "user",
                        "content": latest_message
                    }
                ]
            }
        )
        
        if result.status.value == "completed":
            # Get messages from the thread
            agent_messages = agents_client.messages.list(thread_id=result.thread_id)
            
            # Find the agent's response
            for msg in agent_messages:
                if msg.role.value == "agent":
                    formatted_response = {
                        "content": msg.content[0].text.value,
                        "role": "assistant"
                    }
                    return {"messages": [formatted_response]}
        
        # If no agent response found or run failed
        error_response = {
            "content": f"I encountered an error processing your request. Status: {result.status.value}",
            "role": "assistant"
        }
        return {"messages": [error_response]}
        
    except Exception as e:
        error_response = {
            "content": f"I encountered an error and couldn't process your request: {str(e)}",
            "role": "assistant"
        }
        return {"messages": [error_response]}

print("Restaurant agent callback function defined!")

Restaurant agent callback function defined!


## Initialize Red Team Evaluator

In [ ]:
# Azure AI project endpoint for red team
azure_ai_project = RESTAURANT_ASSISTANT_PROJECT

# Create the RedTeam instance with specific risk categories relevant to a restaurant agent
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm
    ],
    num_objectives=3,  # Number of attack objectives per risk category
)

print("Red Team evaluator initialized!")
#print(f"Risk categories: {[cat.value for cat in red_team._risk_categories]}")
#print(f"Number of objectives per category: {red_team._num_objectives}")

Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Red Team evaluator initialized!


AttributeError: 'RedTeam' object has no attribute '_risk_categories'

## Run Basic Red Team Scan

This performs a basic red team evaluation using simple attack strategies.

In [ ]:
# Run a basic red team scan
print("Starting basic red team scan...")
print("This may take several minutes to complete.")

basic_result = await red_team.scan(
    target=restaurant_agent_callback,
    scan_name="Restaurant-Agent-Basic-Red-Team",
    # Baseline attack strategy only (baseline is included by default)
    output_path="basic_red_team_results.json",
)

print("Basic red team scan completed!")
print("Results saved to: basic_red_team_results.json")

## Run Advanced Red Team Scan

This performs a more comprehensive red team evaluation with multiple attack strategies.

In [ ]:
# Run an advanced red team scan mirroring the sample's strategies
print("Starting advanced red team scan with comprehensive strategies...")

advanced_result = await red_team.scan(
    target=restaurant_agent_callback,
    scan_name="Restaurant-Agent-Advanced-Red-Team",
    attack_strategies=[
        AttackStrategy.Base64,
        AttackStrategy.Flip,
        AttackStrategy.CharacterSpace,
        AttackStrategy.ROT13,
        AttackStrategy.UnicodeConfusable,
        AttackStrategy.Tense,
        AttackStrategy.CharSwap,
        AttackStrategy.Morse,
        AttackStrategy.Leetspeak,
        AttackStrategy.Url,
        AttackStrategy.Binary,
        AttackStrategy.MODERATE,
        AttackStrategy.DIFFICULT,
    ],
    output_path="advanced_red_team_results.json",
)

print("Advanced red team scan completed!")
print("Results saved to: advanced_red_team_results.json")

In [ ]:
# Quick summary of results
try:
    basic_count = len(basic_result.redteaming_data) if getattr(basic_result, "redteaming_data", None) else 0
except Exception:
    basic_count = 0

try:
    advanced_count = len(advanced_result.redteaming_data) if getattr(advanced_result, "redteaming_data", None) else 0
except Exception:
    advanced_count = 0

print(f"Basic conversations collected: {basic_count}")
print(f"Advanced conversations collected: {advanced_count}")
print("Files: basic_red_team_results.json, advanced_red_team_results.json")